In [ ]:
from bs4 import BeautifulSoup
from fnmatch import fnmatch
from PIL import Image
from PIL import ImageFile
from pathlib import Path
from shutil import copyfile

import imagehash
import mysql.connector
import os
import re

In [ ]:
# GENERAL CONFIGS
ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_cm"


# == Files config ==
MAIN_FOLDER = r"D:\dnmarchives\dnmarchives\cryptomarket"

In [ ]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

cursor.execute("DROP TABLE IF EXISTS products_tmp_cm")


In [ ]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_cm ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [ ]:
def get_content(content):
    clear_content = ""

    if content is not None:
        raw_c = content.text

        clear_content = re.sub(r'[\t\r\n]', ' ', raw_c)
        clear_content = re.sub(' +',' ', clear_content).lstrip()
        
    return clear_content

In [ ]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}")
    
#     try:
    for file in os.listdir(folder):            
        if "item.php_id=" in file:
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                
                try:
                    desc_content = page_content.find("div", attrs={"style": "padding:20px; font-size:14px;"})

                    if desc_content is not None:
                        description = get_content(desc_content)
                except:
                    description = ""

                ## GET TITLE
                title = ""
                
                try:
                    raw_title = page_content.findAll("h3")

                    if raw_title is not None and len(raw_title) > 0:
                        title = get_content(raw_title[0])
                except:
                    title = ""

                if description != "" or title != "":
                    try:
                        # Insert product into database
                        query = "INSERT INTO products_tmp_cm (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
                        values = (title, description, "Cryptomarket", "t")

                        cursor.execute(query, values)
                        conn.commit()

                        product_id = cursor.lastrowid

                        ## GET IMAGE
                        main_div = page_content.find("a", attrs={"target": "_blank"})

                        if main_div is not None:
                            name = main_div["href"]

                            if name is not None:
                                url_image = Path(f"{folder}\{name}")

                                try:
                                    # copy images to new directory, renaming with product id
                                    if url_image.exists():
                                        print(f"Image {product_id} exists... creating")

                                        dst_dir = Path(f"D:\images\_cm\{product_id}.jpeg")
                                        copyfile(url_image, dst_dir)
                                    else:
                                        print(f"Image {product_id} does not exists")
                                except:
                                    print(f"Image {product_id} has error. Except")
                    except:
                        print("Bad description")
#     except:
#         print(f"Error opening file into {folder}")

